<a href="https://colab.research.google.com/github/kspohn903/MegaMillions_ML_RidgeRegression_Predictor/blob/main/MegaMillionsExtractionAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #PACKAGE IMPORTS/DOWNLOADS/FETCH COMMANDS FOR ENVIRONMENT 

!pip install -q timestring 
!pip install -q datetime 
!pip install -q data
!pip install -q api-client # Alternate data collection protocols... 
!pip install -q requests_html # Web scraper packages
!pip install -q bs4 # unneeded if you have a source of api/ data to feed in... else web scrape.
!pip install -q urllib3 # making sure that OSI L6 stuff can read in surface html and parse web documentation/ documents
!pip install -q nltk # data processing, sentiment analysis 
!pip install -q pandas # data frames, reading in data, blah...
!pip install -q numpy # same as scipy. Manipulating tensor metrics, learning, adapting, etc.
!pip install -q imgaug # Necessary if reading in specific google colab/ tensorflow,
!pip install -q folium # usually used with image classifiers
!pip install -q datascience # tensorflow of choice... manipulating for regressional analytics 
!pip install -q scipy
# !pip install -q google-cloud 
!pip install -q rauth
!pip install -q tensorflow

# # !pip install --upgrade numpy # Alternative Updater to numpy... 
# If not Tensorflow or conflicts in package resolution, then this one...

# ARE YOU ON THE TRAIN??? ... THEN GET ON THE TRAIN!!! Edwarrrrrd!!!! 
# File reading likeness... as STRING_IO.(parserToken)
from google.colab import drive, files, auth
drive.mount('/content/drive', force_remount=True) # mounted / forcibly-remounted


     |████████████████████████████████| 51 kB 275 kB/s 
     |████████████████████████████████| 251 kB 9.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 83 kB 1.3 MB/s 
     |████████████████████████████████| 138 kB 47.7 MB/s 
     |████████████████████████████████| 111 kB 54.0 MB/s 
     |████████████████████████████████| 127 kB 53.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 69 kB 3.2 MB/s 
     |████████████████████████████████| 462 kB 5.2 MB/s 
Mounted at /content/drive


In [ ]:

import pandas as pd, numpy as np, sys 

# # dateutil, secrets act as secondary date parser iff datetime fails or threadback issues of parsing dates...
# # optional args/ imports: ----------------------------------------------------
# import dateutil, secrets, bs4 as soup, urllib3, gzip, tenacity 

#-------------------------------------------------------------------------------
import json, requests, csv, traceback as tb, seaborn as sns, scipy as sci
import matplotlib.pyplot as plt, math, gzip, os, io, re, matplotlib.pylab as pl 
import matplotlib.transforms as mt, datetime, time, gc

# from apiclient import (APIClient, paginated, exceptions, retry_request,
#                HeaderAuthentication, JsonResponseHandler, JsonRequestFormatter, endpoint) 

import sklearn
from datetime import timedelta # any other methods...
# from apiclient.retrying import retry_if_api_request_error
from matplotlib.transforms import offset_copy 
from google.colab import files # Stealing/ Uploading/ Downloading to RAM files from my drive... 
from sklearn.linear_model import LinearRegression
from io import StringIO

# # File reading, analytics, dataframe manipulation en masse, datetime checks, garbage thread collection RAM/ NVRAM
# # Regex checking, files collected (requests/ urllib3), sys for filepath oauth checks... file dumps/reading

# # error thread checking, graphical displays, data analytics, plotting s***, you know
# # Everything here is super kosher... 
# from bs4 import BeautifulSoup as bs
from matplotlib.font_manager import FontProperties


In [ ]:
#Derive_API_JSON_FROM_RAPIDAPI_REQUESTS.py
try:
    # Derive Mega Millions Numerical String Data of Winners from 
    today = datetime.datetime.now() # Current day: e.g. (MM-DD-YYYY = 06-11-2021)
    responseMsg = input("What day are you starting? (e.g. 12/31/2019, 10/31/2017); Format: MM/DD/YYYY")
    dateInput = responseMsg.split("/")
    dateInput = [int(el) for el in dateInput]
    while dateInput[0] not in [i for i in range(1,13,1)] or (
          dateInput[1] not in [i for i in range(1,32,1)]) or ( 
          dateInput[2] not in [yr for yr in range(2017, today.year + 1, 1)]) or (
          re.match("/[(\d{1-2})(-/)(\d{1-2})(-/)(\d{2-4})]/g", responseMsg) ):
          
          responseMsg = input("What day are you starting? (e.g. 12/31/2019, 10/31/2017); Format: MM/DD/YYYY")
          dateInput = responseMsg.split("/")
          dateInput = [int(el) for el in dateInput] 

    firstDay = datetime.datetime(year = dateInput[2], month = dateInput[0], day = dateInput[1])
    print(f"firstDay is {firstDay}\n")
    # Oct 31, 2017; 2020 data is set to Dec, 31, 2019 in the above format
    # 
    # Beyond this point back, number winning schemas are inconsistent and 
    # remain pairwise within certain time periods/ intervals  
    
    fDayString = firstDay.strftime("%m %d %Y")
    fDayString = fDayString.replace(" ","_")
    
    todayString = today.strftime("%m %d %Y")
    todayString = todayString.replace(" ", "_")
    
    print(f"fDayString: {fDayString}\ntodayString: {todayString}\n")
    
    # Modify to closest Tuesday/Thursday, based on bi-weekly draws approx. Midnight that day...
    
    yearA = firstDay.year # MM/DD/YYYY
    print(f"yearA is {yearA}\n")
    superDirName = 'drive/MyDrive/CSV_Files/Mega Millions/'
    filePathName = '{}MegaBall_Schema_{}_as_of_{}.csv'.format(superDirName,yearA,todayString) 
    
    # Manually download from the NY State / NYC municipality dataset. Then, 
    # Append any subsequent data from future dates... append all 5 data tables up to speed ...
    # Then download it... 
    # Now from Excel Google Sheets, read in data as normal

    #  print(f"Before reading in from file path, \ 
    #  Steve-Bot is testing file read in from os, and validity of dataframe...\n")
    
    dfMegaMillions = pd.read_csv(filePathName,header=0)
    print("dfMegaMillions is successfully being read in...\n")
    
    print("---------- The column headers ------------\n")
    headerColumns = list(dfMegaMillions.columns.values)
    print(f"Header Columns: {headerColumns}\n")

    drawingDate = dfMegaMillions[headerColumns[0]].values
    firstNum = dfMegaMillions[headerColumns[1]].values
    secondNum = dfMegaMillions[headerColumns[2]].values
    thirdNum = dfMegaMillions[headerColumns[3]].values
    fourthNum = dfMegaMillions[headerColumns[4]].values
    fifthNum = dfMegaMillions[headerColumns[5]].values
    megaBall = dfMegaMillions[headerColumns[6]].values
    megaplier = dfMegaMillions[headerColumns[7]].values

    lenCheck = [len(drawingDate) == el for el in [len(firstNum), len(secondNum), 
                                                 len(thirdNum), len(fourthNum), 
                                                 len(fifthNum), len(megaBall)] ]

    print(f"arrLenCheck:{lenCheck}, allSameSizeVectors: {False not in lenCheck}\n")
   
    #  data = [drawingDate,firstNumber, secondNumber, thirdNumber, 
    #          fourthNumber, fifthNumber, megaBall, megaplier ]

    megaHeadersJSON = [ "First_Number", "Second_Number", "Third_Number", 
                        "Fourth_Number", "Fifth_Number", "Mega_Ball", "Megaplier" ]
    
    #  print(f"DrawingDate:\n{drawingDate}\nFirstNumber:\n{firstNum}\n")
    #  print(f"SecondNumber:\n{secondNum}\nThirdNumber:\n{thirdNum}\n")
    #  print(f"FourthNumber:\n{fourthNum}\nFifthNumber:\n{fifthNum}\n")
    #  print(f"MegaBall:\n{megaBall}\nMegaplier:\n{megaplier}\n")

    jsonFreq = { # By definition first 5 1-70, last one 1-25
    1 : {"{}".format(i): 0 for i in range(1,71,1)}, # 1-70; Ball 1
    2 : {"{}".format(i): 0 for i in range(1,71,1)}, # 1-70; Ball 2
    3 : {"{}".format(i): 0 for i in range(1,71,1)},  # 1-70; Ball 3 
    4 : {"{}".format(i): 0 for i in range(1,71,1)},  # 1-70; Ball 4                                          
    5 : {"{}".format(i): 0 for i in range(1,71,1)}, # 1-70; Ball 5
    6 : {"{}".format(i): 0 for i in range(1,26,1)}, # 1-25; Mega Ball 
    } # append all elements into each respective json
   
    print(f"Before populating by frequencies:\n{jsonFreq}\n")
    for i in range(0, len(firstNum), 1): #dnc about the megaplier
        jsonFreq[1]["{}".format(firstNum[i])] += 1
        jsonFreq[2]["{}".format(secondNum[i])] += 1      
        jsonFreq[3]["{}".format(thirdNum[i])] += 1
        jsonFreq[4]["{}".format(fourthNum[i])] += 1
        jsonFreq[5]["{}".format(fifthNum[i])] += 1
        jsonFreq[6]["{}".format(megaBall[i])] += 1
        
        #  print(f"Populating Frequency Iteration no. {i+1}: (")
        #  print(f"json1={jsonFreq[1]},json2={jsonFreq[2]},json3={jsonFreq[3]}", end=',') 
        # print(f"json4={jsonFreq[4]}, json5={jsonFreq[5]}, json6={jsonFreq[6]})", end = "\n")
    print(f"After: {jsonFreq}\n")
    
    #  t1, t2, t3, t4, t5, t6 = 0, 0, 0, 0, 0, 0
    k1 = list(jsonFreq[1].keys())
    k2 = list(jsonFreq[2].keys())
    k3 = list(jsonFreq[3].keys())
    k4 = list(jsonFreq[4].keys())
    k5 = list(jsonFreq[5].keys())
    k6 = list(jsonFreq[6].keys())
    #  print(f"k1: {k1}\nk2: {k2}\nk3: {k3}\nk4: {k4}\nk5: {k5}\nk6: {k6}\n")
   
    v1 = list(jsonFreq[1].values()) # Frequencies of 1-70 for Nums. 1-5
    v2 = list(jsonFreq[2].values())
    v3 = list(jsonFreq[3].values())
    v4 = list(jsonFreq[4].values()) 
    v5 = list(jsonFreq[5].values())
    v6 = list(jsonFreq[6].values()) # MegaBall Frequencies 1-25 
    #  print(f"v1: {v1}\nv2: {v2}\nv3: {v3}\nv4: {v4}\nv5: {v5}\nv6: {v6}\n")
   
    vTot = [int(el) for el in v1]
    t1, t2, t3, t4, t5, t6 = 0, 0, 0, 0, 0, 0
    for i in range(0,len(v1), 1):
       e1 = int(vTot[i])
       e2 = int(v2[i])
       e3 = int(v3[i])
       e4 = int(v4[i])
       e5 = int(v5[i])
       
       #  print(f"(e1={e1}, e2={e2}, e3={e3}, e4={e4}, e5={e5})\n")
       sumFreq = int (e2+e3+e4+e5)

       #  print(f"Prior to appending sumFreq: (lTot: {int(vTot[i])}, sumFreq:{sumFreq})\n")
       t1 += int(v1[i])
       t2 += int(v2[i])
       t3 += int(v3[i])
       t4 += int(v4[i])
       t5 += int(v5[i])
       vTot[i] += int(sumFreq)
       
       #  print(f"Total Frequency for Digit No.{i+1} (for Balls 1-5), is {int(vTot[i])}.\n")
       #  print(f"Stats for iteration No. = {i+1}: (t1={t1}, t2={t2}, t3={t3}, t4={t4}, t5={t5})\n")
       
       print(f"Final Stats: (t1:{t1}, t2:{t2}, t3:{t3}, t4:{t4}, t5:{t5})\n")
       print(f"After total appendation, vTot: {vTot}\n")
   
       maxNoDraws = t1
       print(f"maxNoDraws: {maxNoDraws}\n")
   
       jsonFreq[7] = {"{}".format(i+1): int(vTot[i]) for i in range(0,len(vTot),1)}
       print(f"Checking jsonFreq[7] patch: {jsonFreq[7]}\n")
       df_1_5 = pd.DataFrame(data={ "First_Number": jsonFreq[1], 
                                    "Second_Number": jsonFreq[2],
                                    "Third_Number": jsonFreq[3],
                                    "Fourth_Number": jsonFreq[4],
                                    "Fifth_Number": jsonFreq[5],
                                    "Total": jsonFreq[7] 
                                  } )
       #  # weighted stat. avg. of total draws, i.e. max. no draws from dt A to dt B (today).
       df_1_5["Prob_Ball1"] = round(df_1_5["First_Number"]/maxNoDraws, 6) 
       df_1_5["Prob_Ball2"] = round(df_1_5["Second_Number"]/maxNoDraws, 6)
       df_1_5["Prob_Ball3"] = round(df_1_5["Third_Number"]/maxNoDraws, 6)
       df_1_5["Prob_Ball4"] = round(df_1_5["Fourth_Number"]/maxNoDraws, 6)
       df_1_5["Prob_Ball5"] = round(df_1_5["Fifth_Number"]/maxNoDraws, 6)
       df_1_5["Prob_Total"] = round(df_1_5["Total"]/maxNoDraws, 6)

       df_1_5.sort_values(by=['Prob_Total','Total'], inplace=True, ascending=False)
       #  print(f"{df_1_5['Prob_Total']}\n{df_1_5['Prob_Ball1']}\n{df_1_5['Prob_Ball2']}\n{df_1_5['Prob_Ball3']}\n{df_1_5['Prob_Ball4']}\n{df_1_5['Prob_Ball5']}\n")
       df_6 = pd.DataFrame(data={"Mega_Ball": jsonFreq[6] })
       
       df_6["Prob_Mega_Ball"] = round(df_6["Mega_Ball"]/maxNoDraws, 6)
       df_6.sort_values(by=['Prob_Mega_Ball','Mega_Ball'], inplace=True, ascending=False)
       #  print(f"df_1_5:\n{df_1_5}\ndf_6:\n{df_6}\nProb_Total:\n{df_1_5['Prob_Total']}\n")

       df_1_5.to_csv('{}Mega_Millions_Ball5_Stats_{}_as_of_{}.csv'.format(superDirName, yearA, todayString), index = True) # yearNo: e.g. {2020,2021}, v. overall
       df_6.to_csv('{}Mega_Millions_Mega_Ball_Stats_{}_as_of_{}.csv'.format(superDirName, yearA, todayString), index=True)
       print(f"dataframes have been saved to misc. csv files...\n For later extraction, analytics, and likelihood estimators...\nShutting down...\n")
   
except IOError: 
       print(f"Failure to properly close/read file (IO). Please check stack trace and try again.\n")
       tb.print_exc()
except EOFError:
       print(f"Failure to properly read/parse file properly or has reached EOF.Please check stack trace and try again.\n")
       tb.print_exc()
except IndentationError: 
       print(f"Failure to properly indent. Please check stack trace and try again.\n")
       tb.print_exc()
except ImportError: 
       print(f"Import has failed. Please check imports and try again.\n")
       tb.print_exc()
except Exception: 
       print(f"General Exception has occurred. Please check stack trace and try again.\n")
       tb.print_exc()
except Error: 
       print(f"General Error has occurred. Please check stack trace and try again.\n")
       tb.print_exc()          

What day are you starting? (e.g. 12/31/2019, 10/31/2017); Format: MM/DD/YYYY10/31/2017
firstDay is 2017-10-31 00:00:00

fDayString: 10_31_2017
todayString: 02_15_2022

yearA is 2017

dfMegaMillions is successfully being read in...

---------- The column headers ------------

Header Columns: ['DrawingDate', 'FirstNumber', 'SecondNumber', 'ThirdNumber', 'FourthNumber', 'FifthNumber', 'MegaBall', 'Megaplier', 'NumberSet']

arrLenCheck:[True, True, True, True, True, True], allSameSizeVectors: True

Before populating by frequencies:
{1: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 0, '25': 0, '26': 0, '27': 0, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 0, '40': 0, '41': 0, '42': 0, '43': 0, '44': 0, '45': 0, '46': 0, '47': 0, '48': 0, '49': 0, '50': 0, '51': 0, '52': 0, '5

In [ ]:
# Prev. File Name: MEGAMILLIONS_Stat_Pictographic.py
# split dataset in train and testing set   
import matplotlib.pyplot as plt

try: 
    #yearA, responseMsg/todayStr ... No longer needed... Input already validated.
    
    filePathName1 = '{}Mega_Millions_Ball5_Stats_{}_as_of_{}.csv'.format(superDirName, yearA, todayString)
    filePathName2 = '{}Mega_Millions_Mega_Ball_Stats_{}_as_of_{}.csv'.format(superDirName, yearA, todayString)
    # print(f"Before reading in from file path, Steve-Bot is testing file read in from os, and validity of dataframe...\n")
    
    dfMegaMillions_Ball5 = pd.read_csv(filePathName1, header=0, nrows=30) # 
    # print("dfMegaMillions_Ball5 is successfully being read in...\n")
    
    headerColumnsBall5 = dfMegaMillions_Ball5.columns.values
    print(f"The Header Columns Ball5: {headerColumnsBall5}\n")
    
    datasetBall5 = dfMegaMillions_Ball5.values
    dfMegaMillions_MegaBall = pd.read_csv(filePathName2,header=0,nrows=15) 
    datasetMB = dfMegaMillions_MegaBall.values
    # print("dfMegaMillions_MegaBall is successfully being read in...\n")
    
    headerColumnsMegaBall = dfMegaMillions_MegaBall.columns.values
    print(f"The Header Columns MegaBall: {headerColumnsMegaBall}\n")
    
    # Values 5 Ball, megaBall: headerColumnsMegaBall[0], headerColumnsBall5[0]
    # 5 Ball: [Value, First_Number, Second_Number, Third_Number, Fourth_Number, Fifth_Number, Total,
    # Prob_Ball1, Prob_Ball2, Prob_Ball3, Prob_Ball4, Prob_Ball5, Prob_Total]
    # Mega Ball: [Ball Value, Freq_Mega_Ball, Prob_Mega_Ball]
    
    # Assuming values in numB5, numMB are dependent on probability of regressional predictions...
    # Where the probability of a number is treated as independent, non-categorical
    xB5 = dfMegaMillions_Ball5[headerColumnsBall5[0]] # Ball Value for B5
    xMB = dfMegaMillions_MegaBall[headerColumnsMegaBall[0]] # Ball Value for MB
    print(f"(xB5: {xB5.values}\nxMB: {xMB.values})\n")
    
    yB1 = dfMegaMillions_Ball5[headerColumnsBall5[-6]] # Prob_Ball1 Column Data
    yB2 = dfMegaMillions_Ball5[headerColumnsBall5[-5]] # Prob_Ball2 ...
    yB3 = dfMegaMillions_Ball5[headerColumnsBall5[-4]] # Prob_Ball3 ...
    yB4 = dfMegaMillions_Ball5[headerColumnsBall5[-3]] # Prob_Ball4 ...
    yB5 = dfMegaMillions_Ball5[headerColumnsBall5[-2]] # Prob_Ball5 ...
    yBTot = dfMegaMillions_Ball5[headerColumnsBall5[-1]] # Prob_Total ...
    
    # yB = [yB1, yB2, yB3, yB4, yB5, yBTot]
    # Prob Ball_5: {Prob_Ball{i+1}, Prob_Total}
    
    yMB = dfMegaMillions_MegaBall[headerColumnsMegaBall[2]]
    print(f"(yB1:{yB1.values}\nyB2:{yB2.values}\nyB3:{yB3.values}\nyB4:{yB4.values}\nyB5:{yB5.values}\nyBTot:{yBTot.values})\n")
    
    # Creating figure & saving relevant graphics data    
    # EX: ---------------------------------------------------------------------
    # Assuming that len(x)==len(y) for datasets 
    # reg = linear_model.LinearRegression()
    # modelB5 = reg.fit(xB5, yB5)
    # modelMB = reg.fit(xMB, yMB)
    
    # reg2 = linear_model.LinearRegression() 
    # reg2.fit(x_poly, y)

    # plt.title("Polynomial Regression")
    # plt.xlabel("Position Level")
    # plt.ylabel("Salary")
    # plt.scatter(x, y, color ='red')
    # plt.plot(x, lin_reg2.predict(poly_reg.fit_transform(x)), color = 'blue')
    # plt.show()


    nSubgroupsBall5 = 70
    nBall5 = 5

    totNoEl = 6
    count = 0
    bar_width = 0.30
    opacity = 0.80

    fig1, ax1 = plt.subplots(figsize=(10,10))  
    # bar_pos_B5 = np.arange(nSubgroupsBall5)  

    bChartB5_1 = fig1.bar(xB5 + count*(totNoEl)*bar_width, yB1, bar_width,
    alpha=opacity, color='orange', label='Ball {}'.format(count+1))
    count += 1
    
    bChartB5_2 = plt.bar(xB5 + count*(totNoEl)*bar_width, yB2, bar_width,
    alpha=opacity, color='blue', label='Ball {}'.format(count+1))
    count += 1

    bChartB5_3 = plt.bar(xB5 + count*(totNoEl)*bar_width, yB3, bar_width,
    alpha=opacity, color='purple', label='Ball {}'.format(count+1))
    count += 1
    
    bChartB5_4 = plt.bar(xB5 + count*(totNoEl)*bar_width, yB4, bar_width,
    alpha=opacity, color='green', label='Ball {}'.format(count+1))
    count += 1
    
    bChartB5_5 = plt.bar(xB5 +count*(totNoEl)*bar_width, yB5, bar_width,
    alpha=opacity, color='red', label='Ball {}'.format(count+1))
    count += 1
    
    bChartB5_Tot = plt.bar(xB5 + count*(totNoEl)*bar_width, yBTot, bar_width,
    alpha=opacity, color='#A14C75', label='Ball Total')
    count += 1

    # x_train_raw_B5, x_test_raw_B5, y_train_B5, y_test_B5 = train_test_split(
    # # Scrape from B5 
        
    # ) # this function will split train and test data set in 75%-25% respectively
    # vector = TfidfVectorizer()
    # x_train_B5 = vector.fit_transform(x_train_raw)
    # x_test_B5 = vector.transform(x_test_raw)
    # model = linear_model.LinearRegression()
    # model.fit(xB5, yBTot)
    # probPredictor = model.predict()
    # r_square_value=model.score(np.array(x_test).reshape(-1,1),np.array(y_test).reshape(-1,1))
    # print(f'r-square value from Linear Regression: {r_square_value}\n')
     
    # Model 1: Ball5 Values based on B5 data xB5, yB5 = P(xB5 for all XB5 (subset member of xB5; ball 1, 2, ..., ) in xB5) 
    plt.xlabel('Ball Value')
    plt.ylabel('P(Ball)')
    plt.title('Bar Chart of Ball 5 Value v. P(Ball 5)')
    # print(f"Testing if tuple arr concatonator works for str bar chart grouped by stat frequency ***BEFORE***...\n")
    plt.xticks(xB5 + totNoEl*bar_width, tuple([str(ball) for ball in xB5]) )
    # print(f"Testing if tuple arr concatonator works for str bar chart grouped by stat frequency ***AFTER***...\n")
    plt.legend()
    plt.tight_layout()
    plt.savefig('MegaMillions_Ball5_BarChartPlot_{}_{}.png'.format(
        yearA,datetime.datetime.now().strftime("%Y %m %d").replace(" ","-") ) )
    plt.show()
    
    # Plot 2: Mega Ball Distinct plot by horizontal and by sorting order... May create subgrouping axes to 
    # predict in sorted fashion, but documentation I read is insufficient to complete that procedure.
    fig2, ax2 = plt.subplots(figsize=(8,8))
    bChartMB = plt.bar(xMB, yMB, bar_width, alpha=opacity, color='purple', label='Mega Ball')
    plt.xlabel('Ball Value')
    plt.ylabel('P(Mega Ball)')
    plt.title('Bar Chart of Ball 5 Value v. P(Mega Ball)')
    # print(f"Testing if tuple arr concatonator works for str bar chart grouped by stat frequency ***BEFORE***...\n")
    plt.xticks(xMB + bar_width, tuple([str(ball) for ball in xMB]) )
    # print(f"Testing if tuple arr concatonator works for str bar chart grouped by stat frequency ***AFTER***...\n")
    plt.legend()
    plt.tight_layout()
    plt.savefig('MegaMillions_MegaBall_BarChartPlot_{}_{}.png'.format(
        yearA,datetime.datetime.now().strftime("%Y %m %d").replace(" ","-") ) )
    plt.show()
    
    # regB1=LinearRegression()  
    # regB1.fit(numB5,p1B5)  
    # y_pre = reg.predict(X_test) 

    ### Intended for Regression _Prediction, either RidgeRegression using lineardense 
        # as matrix fodder data, and trainsets for linear predictors,
        # based on KNN's, but I'm not skilled enough to use a KNN of that 
        # regression predictor form... Not competent enough, yet.  

except IOError: 
       print(f"Failure to properly close/read file (IO). Please check stack trace and try again.\n")
       tb.print_exc()
except EOFError:
       print(f"Failure to properly read/parse file properly or has reached EOF.Please check stack trace and try again.\n")
       tb.print_exc()
except IndentationError: 
       print(f"Failure to properly indent. Please check stack trace and try again.\n")
       tb.print_exc()
except ImportError: 
       print(f"Import has failed. Please check imports and try again.\n")
       tb.print_exc()
except Exception: 
       print(f"General Exception has occurred. Please check stack trace and try again.\n")
       tb.print_exc()
except Error: 
       print(f"General Error has occurred. Please check stack trace and try again.\n")
       tb.print_exc()


In [ ]:
# Previous Super File Name: ML_MEGAMILLIONS_WHEEL_GENERATOR.ipynb
try:
    # todayFileStr = datetime.datetime.today().strftime("%m %d %Y").replace(" ","_")
    filePathName1 = 'drive/MyDrive/CSV_Files/Mega_Millions_Ball5_Stats_{}_as_of_{}.csv'.format(yearA, todayString)
    filePathName2 = 'drive/MyDrive/CSV_Files/Mega_Millions_Mega_Ball_Stats_{}_as_of_{}.csv'.format(yearA, todayString)
    # print(f"Before reading in from file path, Steve-Bot is testing file read in from os, and validity of dataframe...\n")
    
    dfMegaMillions_Ball5 = pd.read_csv(filePathName1,header=0)
    print("dfMegaMillions_Ball5 is successfully being read in...\n")
    headerColumnsBall5 = dfMegaMillions_Ball5.columns.values
    # print(f"The Header Columns Ball5: {headerColumnsBall5}\n")
    dfMegaMillions_MegaBall = pd.read_csv(filePathName2,header=0)
    print("dfMegaMillions_MegaBall is successfully being read in...\n")

    headerColumnsMegaBall = dfMegaMillions_MegaBall.columns.values
    print(f"The Header Columns MegaBall: {headerColumnsMegaBall}\n")
    
    # print(f"dfs's:\nBall5\n{dfMegaMillions_Ball5}\nMegaBall\n{dfMegaMillions_MegaBall}\n")   
    
    # Verify that each Prob_Mega_Ball for df2, Prob_Total from df1 yield 1 for each column set...
    # Attempt to justify that total probability for each Ball No. 
    megaBallNos = dfMegaMillions_MegaBall[headerColumnsMegaBall[0]].values
    probTotal_Vector = dfMegaMillions_Ball5["Prob_Total"].values
    probAvgTotal_Vector = [el/5 for el in probTotal_Vector]
    
    # print(f"probAvgTotal_Vector: {probAvgTotal_Vector}\n")
    probMegaBall_Vector = dfMegaMillions_MegaBall["Prob_Mega_Ball"].values

    t1, t2 = 0, 0
    # print(f"probTotal_Vector:\n {probTotal_Vector}\nprobMegaBall_Vector:\n {probMegaBall_Vector}\n") 
    
    for i in range(0,len(probTotal_Vector), 1):
        t1 += probTotal_Vector[i]    
    t1 /= 5
    
    for j in range(0, len(probMegaBall_Vector), 1):
        t2 += probMegaBall_Vector[j]
    
    print(f"[b5,mB]: [{t1},{t2}]\n")  
    [pBall1_Vector, pBall2_Vector, pBall3_Vector, pBall4_Vector, pBall5_Vector, ball5Nos] = [
    dfMegaMillions_Ball5["Prob_Ball1"].values, dfMegaMillions_Ball5["Prob_Ball2"].values,
    dfMegaMillions_Ball5["Prob_Ball3"].values, dfMegaMillions_Ball5["Prob_Ball4"].values,
    dfMegaMillions_Ball5["Prob_Ball5"].values, dfMegaMillions_Ball5[headerColumnsBall5[0]].values]
    
    p1,p2,p3,p4,p5 = 0,0,0,0,0
    for k in range(0,len(pBall1_Vector),1):
        p1 += pBall1_Vector[k]
        p2 += pBall2_Vector[k]
        p3 += pBall3_Vector[k]
        p4 += pBall4_Vector[k]
        p5 += pBall5_Vector[k]   

    megaBallNos = megaBallNos[0:10]
    ball5Nos = ball5Nos[0:50] # 6 * (noWheelStructsDesired=; nWSD <= len(vector) && %6==0)-1
    
    # print(f"The top 6 values for ball5Nos and megaBallNos are ({ball5Nos}, {megaBallNos})\n")
    # print(f"[len(ball5Nos),len(megaBallNos)] = [{len(ball5Nos)},{len(megaBallNos)}]\n")    
    
    # ball5Nos = sorted(ball5Nos)
    # megaBallNos = sorted(megaBallNos)
    # print(f"sorted BallNos= (ball5Nos,megaBallNos): ({ball5Nos, megaBallNos})\n")
    
    # print(f"pBall1: {pBall1_Vector}\npBall2: {pBall2_Vector}\npBall3: {pBall3_Vector}\npBall4: {pBall4_Vector}\npBall5: {pBall5_Vector}\n")
    # print(f"probTotal_Vector:\n {probTotal_Vector}\nprobMegaBall_Vector:\n {probMegaBall_Vector}\n") 
    
    # print(f"Hot Numbers by 5 Ball/ Mega Ball, respectively...\nBall5:\n{ball5Nos}\nMega Ball:\n{megaBallNos}\n")
    
    predicted_avg = (p1+p2+p3+p4+p5)/5
    print(f"[p1,p2,p3,p4,p5,predictedAvg]: [{p1}, {p2}, {p3}, {p4}, {p5}, {predicted_avg}]\n")
    
    # Missing probability of at most, +0.06937394247, throughout df[...]_Ball5
    # So, ~100% of all probabilities accounted for... Some lost in rounding, I guess
    # **gaining**, on avg., 0.018849e-2% per entry... 
    # Dunno if that's true or not, but I'll assume yes...

    wheel_data_trainset_ABSPATHNAME= '{}MEGA_MILLIONS_WHEEL_TRAINSET_{}_{}.csv'.format(
        superDirName,yearA,todayString)
    train_set_wheel_numerical = open(wheel_data_trainset_ABSPATHNAME,mode="w+")
    for i in range(1,7,1): # Writing Headers Properly... n%6 == 0 1st iteration only, NOT to 12 mod 6
        if i%6 != 0: # 0-5 
           train_set_wheel_numerical.write("Ball {}, ".format(i%6))
        else: 
           train_set_wheel_numerical.write("Mega Ball\n")

    wheelSchema = { "Set 1": [1,2,3,4,5,6,7,8,9,10,11,12],
                    "Set 2":[1,3,5,7,9,11,2,4,6,8,10,12], 
                    "Set 3":[1,2,3,10,11,12,4,5,6,7,8,9],  
                    "Set 4":[1,2,3,7,8,9,4,5,6,10,11,12] }
    noWheelSets = 4
    for n in range(0, int(len(ball5Nos)/10), 1):
        # print(f"Iteration no. for n: {n+1}\n")
        for j in range(0, int(len(megaBallNos)/5),1):
            b1_1 = ball5Nos[10*n]
            b2_1 = ball5Nos[10*n+1]
            b3_1 = ball5Nos[10*n+2]
            b4_1 = ball5Nos[10*n+3]
            b5_1 = ball5Nos[10*n+4]
            mB1 = megaBallNos[j]
            b1_2 = ball5Nos[10*n+5]
            b2_2 = ball5Nos[10*n+6]
            b3_2 = ball5Nos[10*n+7]
            b4_2 = ball5Nos[10*n+8]
            b5_2 = ball5Nos[10*n+9]
            mB2 = megaBallNos[j+1]
            wheelSet = [b1_1, b2_1, b3_1, b4_1, b5_1, mB1,
                        b1_2, b2_2, b3_2, b4_2, b5_2, mB2]

            for m in range(1,noWheelSets+1,1):
                # print(f"Set No. {m} ")
                setNo = wheelSchema["Set {}".format(m)]
                for k in range(0, len(setNo),1):
                    if (k+1)%6 != 0:
                      #  print(f"{wheelSet[setNo[k]-1]} ")
                      train_set_wheel_numerical.write("{}, ".format(wheelSet[setNo[k]-1]))
                    else: 
                      #  print(f"{wheelSet[setNo[k]-1]}\n")
                      mB = [mB1,mB2]
                      train_set_wheel_numerical.write("{}\n".format(wheelSet[setNo[k]-1] if wheelSet[setNo[k]-1] < 26 else mB[j]))
                # print(f"Set No. {m} Printed successfully...\n")
            # print(f"Internal iteration no., j is {j+1}\n")
        # print(f"End of iteration no. {n+1}...\n")
        print(f"Garbage Dump! .. Please stand *urp* back...\n")
        gc.collect()
        # print(f"Forcibly reallocating memory at end of loop iteration no. {n+1}\n")     
    print(f"Closing train_set_wheel_numerical...\n")
    train_set_wheel_numerical.close()

except IOError: 
       print(f"Failure to properly close/read file (IO). Please check stack trace and try again.\n")
       tb.print_exc()
except EOFError:
       print(f"Failure to properly read/parse file properly or has reached EOF.Please check stack trace and try again.\n")
       tb.print_exc()
except IndentationError: 
       print(f"Failure to properly indent. Please check stack trace and try again.\n")
       tb.print_exc()
except ImportError: 
       print(f"Import has failed. Please check imports and try again.\n")
       tb.print_exc()
except Exception: 
       print(f"General Exception has occurred. Please check stack trace and try again.\n")
       tb.print_exc()
except Error: 
       print(f"General Error has occurred. Please check stack trace and try again.\n")
       tb.print_exc()   

Failure to properly close/read file (IO). Please check stack trace and try again.



Traceback (most recent call last):
  File "<ipython-input-32-edcb6bb2ab02>", line 9, in <module>
    dfMegaMillions_Ball5 = pd.read_csv(filePathName1,header=0)
  File "/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/parsers/readers.py", line 586, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/parsers/readers.py", line 482, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/parsers/readers.py", line 811, in __init__
    self._engine = self._make_engine(self.engine)
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/parsers/readers.py", line 1040, in _make_engine
    return mapping[engine](self.f, **self.options)  # type: ignore[call-arg]
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/parsers/c_parser_wrapper.

In [ ]:
# Prize Validator
# Check your Mega Millions/ PowerBall numbers!

# orginal checker app code created by Brandon I.
# URL Brandon I: https://github.com/Und3rf10w/Powerball-results-checker

import sys, io, json, pandas as pd, numpy as np
import scipy as sci
wheelFilePath = "{}MEGA_MILLIONS_WHEEL_TRAINSET_{}_{}.csv".format(superDirName,
                                                             yearA,todayString)
schemaFilePath = "{}MegaBall_Schema_{}_as_of_{}.csv".format(superDirName,yearA, todayString)

input_winning_numbers = open(schemaFilePath,"r+") #opens winning numbers text file in same directory as script file.
# format the file with a comma delimeter, for example: 19,25,29,36,48,12
# the first five should be the white numbers and the last (sixth) should be the powerball
input_your_numbers = open(wheelFilePath,"r+") 
# opens the text file containing your numbers on your ticket in the same directory as script file.
# format the file with a comma delimeter as above, but a semicolon between sets of numbers on the ticket
# for example: 19,25,29,36,48,12;5,6,29,35,51,21;34,39,42,44,59,8

initial_winning_numbers = input_winning_numbers.read() # reads the text file
initial_your_numbers = input_your_numbers.read() # reads the text file

winning_numbers = initial_winning_numbers.split(',') #splits the winning numbers into a list
#now we will make sure our inputted numbers are actually integers. 
# it will try what is in try, but if there is an error (or what python calls an exception), 
# then run the code under except
try:    
    sanitized_win = []
    for number in winning_numbers:
        tempwin = int(number)
        sanitized_win.append(tempwin)
except:
    print("There seems to be an error with the winning numbers file. Are they numbers?")
    input("Press Enter to Exit")
    sys.exit()

#this will be how we check our numbers
def checknumbers(sanitized_nums):
    global sanitized_win
    whiteBalls = 0 #will count how many white balls we match
    matchPowerBall = False
    if sanitized_nums[5] == sanitized_win[5]: #checks winningness of powerball
        matchpowerball = True
        print("You matched the powerball")
    else:
        print("You did not match the powerball")
    
    myNumbers = [sanitized_nums[i] for i in range(0,5,1)]
    winWhites = [sanitized_win[i] for i in range(0,5,1)]
    for myNum in myNumbers: #will check numbers for matches
        for winNum in winWhites:
            if myNum == winNum:
               whiteBalls += 1
    print ("You matched {} White Balls...\n".format(whiteBalls))
    #check for prizes
    
    powerBallPrizes = { "whiteBalls_by_matchPowerBall": { "True": [4, 4, 7, 100, 50000, "Jackpot"], 
                                                          "False": [0, 0, 0, 7, 100, 1000000]  } 
    }
    prize = powerBallPrizes["whiteBalls_by_matchPowerBall"]["{}".format(matchPowerBall)][whiteBalls]
    return str(prize)

separate_tickets = initial_your_numbers.split(';') #splits tickets via the semicolon delimeter

ticketCount = 0
for ticket in separate_tickets: #run this for each ticket in our list
    ticketCount += 1 #will identify our ticket by number
    numbers = ticket.split(',') #separates the ticket's numbers
    print(f"Ticket {ticketCount}")
    #this works the same as the try except above, pulling from the list of 'numbers'
    try:
       sanitizedNums = []
       for number in numbers:
           tempwin = int(number)
           sanitizedNums.append(tempwin)
           # this sends the numbers to my check 'function' above and returns the prize amount
           ticketprize = checknumbers(sanitizedNums)
           print("The prize for ticket no. {} is ${}.\n".format(ticketCount, ticketprize) )
    
    except:
        print(f"There seems to be an error with ticket {ticketCount}. Are they numbers?")
    finally:
        print("")

#closes the files correctly
input_winning_numbers.close()
input_your_numbers.close()
print (""" Prizes differ in California. See powerball.com for CA prize info. This program and its developer 
           are not related in any way to Powerball or the Multistate Lottery Association (MUSL). 
           Numbers not official until verified.
       """)
# input("Press enter to exit")

In [ ]:
# ML Input Validator on Prize Value and Vectorizer Loss / Probability Expectation on Ridge-Regression Model-Type

# !pip install sklearn
# !pip install scipy
# !pip install scikit-learn

from sklearn import datasets, linear_model, svm # linear_model.(LinearRegression,LogisticRegression) 
# Preferential Treatment to LogisticRegression, RidgeRegression (aka KNN-LogRegression) 
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split 
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler